# `where` are the `layerDefs`?

Currently there is no official api published on how to further filter the result list returned by `/identify`. There is a working implementation using the query parameter `where`, which is used internally by the viewer, and maybe also externally (to be clarified). This query parameter is however not documented. Upon two requests from clients there have been made attempts to improve and stabilize the possibility to filter. In the current WIP, two query params are used that use exactly the same parsing logic in the background, but different formats in the query, `where` and `layerDefs`. Ideally, we'd just making one of them publicly available (and documented). `layerDefs` offers to extend the current functionality in the future (cross-layer filtering) with keeping the same syntax so it's the preferred solution. You find below an example on how to use `layerDefs` using python.

## Queryable attributes
Not all attributes of a model are queryable. The example below shows how to find the ones that can be used to further filter the results found by identify or find:
```python
>>> import requests
>>> res=requests.get("https://mf-chsdi3.int.bgdi.ch/diemo/rest/services/all/MapServer/ch.swisstopo.amtliches-strassenverzeichnis")
>>> print(', '.join(['{} ({})'.format(field['name'], field['type']) for field in res.json()['fields']]))
plzo (VARCHAR), type (VARCHAR), gdenr (INTEGER), gdename (VARCHAR), label (VARCHAR)
```

## layerDefs syntax
The syntax of the `layerDefs` parameter is a json with the layername as key and the filter expression as value:
```
{"<layername>":"<filter_expression>"}
```
The filter expression can consist of a single expression of the form `<attribute> <operator> <value>` or several of these expressions combined with boolean operators `and` and `or`. `<attribute>` mus be one of the queryable attributes, the type of `<value>` must correspond the the type of the queryable attribute (see above) and `<operator>` can be one of `"<="|">="|"<"|">"|"!="|"="|"ilike"|"not ilike"|"not like"|"like"`. 


## Correct encoding
It's important, that the parameters are correctly serialized and url-encoded, e.g.
```python
>>> import json
>>> import urllib.parse
>>> params = {
        "ch.swisstopo.amtliches-strassenverzeichnis": "plzo = '8302 Kloten'"
    }
>>> print(json.dumps(params))
{"ch.swisstopo.amtliches-strassenverzeichnis": "plzo = '8302 Kloten'"}
>>> print(urllib.parse.quote(json.dumps(params)))
%7B%22ch.swisstopo.amtliches-strassenverzeichnis%22%3A%20%22plzo%20%3D%20%278302%20Kloten%27%22%7D
```

## Find and Identify
The described `layerDefs` syntax is available for both `/find` and `/identify` to subfilter the primary result query. The difference between `/find` and `/identify` is that the primary query is based on a spatial restriction whereas for `/find` the primary query is already defined by an attribute restriction.

## Examples

In [306]:
import requests
import pprint
import urllib.parse

####################################
# A list of examples to demonstrate usage
identify_examples = {
    "ch.bazl.luftfahrthindernis": [
        "bgdi_activesince <= '2019-07-30'"
    ],
    "ch.swisstopo.amtliches-strassenverzeichnis": [
        "gdenr << 2704", # should not work
        "gdenr != 2704",
        "plzo ilike '%Olten%' and label ilike '%Studerweg%'",
        "plzo ilike '%Olten%' and label ilike '%Studerwe'", # should return no results
        "plzo ilike '%Olten%' or label ilike '%Studerweg%'",
        "plzo ilike '%Olten%' OR label ilike '%Studerweg%'",
        "label = 'Studerweg' and plzo not ilike '%Olten%'",
        "plzo = '8302 Kloten'",
        "plzo ilike '%Murten%'",
        "plzo ilike '%Basel%'",
        "plzo ilike '8302%'",
        "blah ilike '8302%'", # should not work
        "gdename ilike 'sel%'",
        "label ilike '%Hauptstrasse%'",
        "label ilike 'Im Heimatla%'", # 'Im Heimatland' in Basel
        "gdenr > 2701 and gdenr < 2704",
        "gdenr > 2701 and gdenr < 2704 and gdename = 'Seltisberg'", # should not return any results
    ],
    "ch.bfe.ladestellen-elektromobilitaet": [
        "IsOpen24Hours = true",
        "IsOpen24Hours is true",
        "QueryAuthenticationModes ilike '%nfc%'", 
        "Longitude > 7.476",
        "QueryAuthenticationModes ilike '%nfc%' and Longitude > 7.476",
    ]
}

find_examples = {
    "ch.swisstopo.amtliches-strassenverzeichnis": [
#         {"searchText": "Studerweg", "searchField": "label", "layerDefs": "plzo ilike '%Olten%'"},
    ]
}

BRANCH = 'layerdefs'

####################################
# Helpers
def encode_param(query):
    return urllib.parse.quote(query)

def decode_param(query):
    return urllib.parse.unquote(query)

def get_identify_url(params):
    """ params needs to be a dict of the form
    params = {
        "branch":'',
        "layer":'ch.swisstopo.amtliches-strassenverzeichnis',
        "paramName":'layerDefs',
        "paramValue":"plzo+=+'8302 Kloten'"
    }
    """
    _url = """https://mf-chsdi3.dev.bgdi.ch/
{branch}/
rest/services/all/MapServer/
identify?
geometry=2577070,1184860
&geometryType=esriGeometryPoint
&geometryFormat=geojson
&tolerance=500000
&mapExtent=0,0,100,100
&imageDisplay=100,100,100
&lang=de
&layers=all:{layer}
&returnGeometry=false
&sr=2056
&{paramName}={paramValue}
    """
    url = _url.format(**params).strip().replace('\n','')
    return url

def get_find_url(params):
    """ params needs to be a dict of the form
    params = {
        "branch":'',
        "layer":'ch.swisstopo.amtliches-strassenverzeichnis',
        "paramName":'layerDefs',
        "paramValue":"plzo+=+'8302 Kloten'"
    }
    """
    _url = """https://mf-chsdi3.dev.bgdi.ch/
{branch}/
rest/services/all/MapServer/
find?
layer={layer}
&searchText={searchText}
&searchField={searchField}
&returnGeometry=false
&contains=false
&{paramName}={paramValue}
    """
    url = _url.format(**params).strip().replace('\n','')
    return url
    
def get_url(variant, params):
    if variant == 'identify':
        url = get_identify_url(params)
    else:
        url = get_find_url(params)
    print("\n=================\nrequesting {}".format(decode_param(params['paramValue'])))
#     print(url)
    response = requests.get(url)
    if response.status_code == 200:
        results = response.json().get('results', [])
        print("--> success! {} result(s)".format(len(results)))

        # print the first result
        pprint.pprint(next(iter(results), []))

    else:
        print("--> example :{}: returned error {}: {}".format(example, response.status_code, response.text))

        
# for layer, layer_examples in identify_examples.items():
#     for example in layer_examples:
#         layerDefsDict = {layer:example}
#         layerDefs_params = {
#             "branch":BRANCH,
#             "layer":layer,
#             "paramName":'layerDefs',
#             "paramValue":encode_param(json.dumps(layerDefsDict))
#         }
#         print(get_identify_url(layerDefs_params))

### Exceute requests

In [300]:
import json

for layer, layer_examples in identify_examples.items():
    for example in layer_examples:
        layerDefsDict = {layer:example}
        layerDefs_params = {
            "branch":BRANCH,
            "layer":layer,
            "paramName":'layerDefs',
            "paramValue":encode_param(json.dumps(layerDefsDict))
        }
        get_url('identify',layerDefs_params)

for layer, layer_examples in find_examples.items():
    for example in layer_examples:
        layerDefsDict = {layer:example['layerDefs']}
        layerDefs_params = {
            "branch":BRANCH,
            "layer":layer,
            "paramName":'layerDefs',
            "searchText": example['searchText'],
            "searchField": example['searchField'],            
            "paramValue":encode_param(json.dumps(layerDefsDict))
        }
        get_url('find',layerDefs_params)


requesting {"ch.bazl.luftfahrthindernis": "bgdi_activesince <= '2019-07-30'"}
--> success! 201 result(s)
{'featureId': 1881,
 'id': 1881,
 'layerBodId': 'ch.bazl.luftfahrthindernis',
 'layerName': 'Luftfahrthindernisse',
 'properties': {'abortionaccomplished': None,
                'bgdi_activesince': '1950-01-01',
                'bgdi_created': '19.08.2019',
                'duration': 'PERM',
                'geomtype': 'line',
                'label': '236-HL-26',
                'lk100': '33',
                'maxheightagl': 60,
                'obstacletype': 'Power line',
                'registrationnumber': '236-HL-26',
                'sanctiontext': 'No Marking and/or Lighting',
                'startofconstruction': '1950-01-01',
                'state': 'A',
                'topelevationamsl': 422,
                'totallength': 97}}

requesting {"ch.swisstopo.amtliches-strassenverzeichnis": "gdenr << 2704"}
--> example :gdenr << 2704: returned error 400: {"status":"error

## where
**Note: the `where` parameter is DEPRECATED! Use layerdefs instead**

When using `where` as query param, the filter expression with the attribute(s) contained in the layer specified for identify are directly passed, e.g.
```
&where=plzo+=+'8302 Kloten'
```

### Examples

In [276]:
for layer, layer_examples in identify_examples.items():
    for example in layer_examples:
        where_params = {
            "branch":BRANCH,
            "paramName":'where',
            "layer":layer,
            "paramValue":encode_param(example)
        }
        get_url('identify',where_params)


requesting bgdi_activesince <= '2019-07-30'
--> success! 201 result(s)
{'featureId': 1881,
 'id': 1881,
 'layerBodId': 'ch.bazl.luftfahrthindernis',
 'layerName': 'Luftfahrthindernisse',
 'properties': {'abortionaccomplished': None,
                'bgdi_activesince': '1950-01-01',
                'bgdi_created': '19.08.2019',
                'duration': 'PERM',
                'geomtype': 'line',
                'label': '236-HL-26',
                'lk100': '33',
                'maxheightagl': 60,
                'obstacletype': 'Power line',
                'registrationnumber': '236-HL-26',
                'sanctiontext': 'No Marking and/or Lighting',
                'startofconstruction': '1950-01-01',
                'state': 'A',
                'topelevationamsl': 422,
                'totallength': 97}}

requesting plzo ilike '%Olten%' and label ilike '%Studerweg%'
--> success! 1 result(s)
{'featureId': 6569342,
 'id': 6569342,
 'layerBodId': 'ch.swisstopo.amtliches-strassen

## sqlinjection risk

This kind of filtering possibly opens the door for sql injections. The current implementation (which is the same for where and layerDefs) uses a strict and restrictive grammer to parse allowed expressions. The following cases must not work:

In [277]:
sqlinj = {
    "ch.swisstopo.amtliches-strassenverzeichnis": [
        "plzo ilike '%Olten%' and 1=1",
        "1=1",
        "true=true"
    ]
}
for layer, layer_examples in sqlinj.items():
    for example in layer_examples:
        where_params = {
            "branch":BRANCH,
            "paramName":'where',
            "layer":layer,
            "paramValue":encode_param(example)
        }
        get_url('identify',where_params)


requesting plzo ilike '%Olten%' and 1=1
--> example :plzo ilike '%Olten%' and 1=1: returned error 400: {"status":"error","code":400,"detail":"The where/layerDefs clause is not valid for ch.swisstopo.amtliches-strassenverzeichnis."}

requesting 1=1
--> example :1=1: returned error 400: {"status":"error","code":400,"detail":"The where/layerDefs clause is not valid for ch.swisstopo.amtliches-strassenverzeichnis."}

requesting true=true
--> example :true=true: returned error 400: {"status":"error","code":400,"detail":"Query attribute 'true' is not queryable. Queryable attributes are 'label,plzo,gdename,gdenr,type'"}


### Open questions


- who is using where other than frontend?
- how to configure in bod to identify layer that is visualized as json but queryable?
- street number? (not included in label, included at all in strassenverz.?)

